#Public Highlights
##Python Code
* **About the Data**
    * Got data from userhighlights not teamhighlights
    * All userhighlights = 7286510
    * Userhighlights with views = 6927719
    * *5% of deleted because data no v for view*
* **More information**
    * mean: 24.9429066046
    * stdev: 505.207633089
    * max: 1056352.0
    * min: 0.0
    * median: 4.0
        * *after removing 0 views median is 6*
* **Zero Views**
    * 1134396 highlights
        * *16% of all userhighlights*
    * 99.8% of the 0 views are non premium highlights
        * 18% of all non premium highlights are 0 views
    * .3% of all premium highlights are 0 views
        * .2 % of the 0 views are non premium highlights
    

In [10]:
import json
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import numpy
from plotly import tools
import time
plotly.tools.set_credentials_file(username='pujaarajan', api_key='7m16vvaw4j')

In [3]:
def reject_outliers(data):
    third_quartile = numpy.percentile(data, 75)
    first_quartile = numpy.percentile(data, 25)
    outlier = (third_quartile - first_quartile)*3
    lower_limit = first_quartile - outlier
    upper_limit = third_quartile + outlier
    return [x for x in data if x < upper_limit and x > lower_limit]

In [ ]:
def reject_xoutliers(data):
    x_data = [x[0] for x in data]
    third_quartile = numpy.percentile(x_data, 75)
    first_quartile = numpy.percentile(x_data, 25)
    outlier = (third_quartile - first_quartile)*3
    lower_limit = first_quartile - outlier
    upper_limit = third_quartile + outlier
    new_data = []
    x_count = 0
    for x in x_data:
        if x < 0:
            x_count = x_count + 1
        elif x_data[x_count] < upper_limit and x_data[x_count] > lower_limit:
            new_data.append(data[x_count])
            x_count = x_count + 1
        else:
            x_count = x_count + 1
    return new_data

def reject_youtliers(data):
    y_data = [y[1] for y in data]
    third_quartile = numpy.percentile(y_data, 75)
    first_quartile = numpy.percentile(y_data, 25)
    outlier = (third_quartile - first_quartile)*3
    lower_limit = first_quartile - outlier
    upper_limit = third_quartile + outlier
    new_data = []
    y_count = 0
    for y in y_data:
        if y_data[y_count] < upper_limit and y_data[y_count] > lower_limit:
            new_data.append(data[y_count])
            y_count = y_count + 1
        else:
            y_count = y_count + 1
    return new_data

In [4]:
sqlc = HiveContext(sc)
user_highlights = sc.textFile("s3://ds-fulla/mongo/20150507/monolith/highlights/userhighlights/").map(json.loads)
highlight_views = user_highlights.filter(lambda x: 'v' in x.keys())

In [7]:
histogram = highlight_views.map(lambda x: x['v']).collect()

In [16]:
data = Data([Histogram(x=histogram)])

layout = dict(
   title='Public Highlight Analysis',
   yaxis=YAxis(
       title = 'Frequency'),
   xaxis1=XAxis(
       title = "Number of Highlight Views", range = [0,200])
   )
fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [25]:
all_views = highlight_views.map(lambda x: x['v']).reduce(lambda a,b: a+b)
zero_views = highlight_views.filter(lambda x: x['v']==0)
highlight_views.map(lambda x: x['v']).stats()
yz_median = numpy.median(highlight_views.map(lambda x: x['v']).collect())
nz_median = numpy.median(highlight_views.filter(lambda x: x['v']!=0).map(lambda x: x['v']).collect())

4.0

In [19]:
sc.parallelize(reject_outliers(histogram)).stats()

(count: 6316783, mean: 6.84204443939, stdev: 8.66066699361, max: 44.0, min: 0.0)

In [21]:
numpy.median(reject_outliers(histogram))

3.0

In [20]:
highlight_views.map(lambda x: x['v']).stats()


(count: 6927719, mean: 24.9429066046, stdev: 505.207633089, max: 1056352.0, min: 0.0)

##Premium
* Premium highlights = 636552 
* Premium highlight views = 114767406  
* Premium view information
    * mean: 180.295413415
    * stdev: 1652.72230355
    * max: 1056352.0
    * min: 0.0
    * median: 72

In [4]:
yp_map = highlight_views.filter(lambda x: x['p']).map(lambda x: x['v'])
yp_reduce = yp_map.reduce(lambda a,b: a+b)
yp_views = yp_map.collect()
yp_median = numpy.median(yp_views)
yp_map.stats()

(count: 636552, mean: 180.295413415, stdev: 1652.72230355, max: 1056352.0, min: 0.0)

In [ ]:
zyp_views = zero_views.filter(lambda x: x['p']).count()
nzyp_map = yp_map.filter(lambda x: x!=0)
nzyp_reduce = nzyp_map.reduce(lambda a,b: a+b)
nzyp_views = nzyp_map.collect()
nzyp_median = numpy.median(nzyp_views)
nzyp_map.stats()

In [ ]:
no_yp = reject_outliers(yp_views)

In [29]:
data = Data([
    Bar(
        y=[x/float(114767406) for x in yp_views],
        x=[x for x in yp_views],
        name='Relative Frequency')])
        #autobinx=True,
        #xbins=XBins(start=min(x),end=max(x),size=1)
        

layout = Layout(xaxis=XAxis(type='log',title = "Number of Premium Highlight Views")
                ,yaxis=YAxis(title = "Frequency"))

fig = Figure(data = data, layout = layout)
py.iplot(fig)

The draw time for this plot will be slow for all clients.


##Not premium
* Not premium highlights = 6291167
* Not premium highlight views = 58030042 
* Not premium view information
    * mean: 9.22405048221
    * stdev: 44.6484461505
    * max: 71279.0
    * min: 0.0
    * median: 3

In [13]:
np_map = highlight_views.filter(lambda x: x['p'] == False).map(lambda x: x['v'])
np_zero = zero_views.filter(lambda x: x['p'] == False).count()
np_reduce = np_map.reduce(lambda a,b: a+b)
np_views = np_map.collect()
np_median = numpy.median(np_views)
np_map.stats()

(count: 6291167, mean: 9.22405048221, stdev: 44.6484461505, max: 71279.0, min: 0.0)

##Spot Shadows
* Highlights with spot shadows = 1668958
    * *1673769 before removing hightlights without views*
    * *.2% data lost*
* Highlights with spot shadows views = 82279191 
* Highlights with spot shadows view information
    * mean: 49.2997373211
    * stdev: 557.941787933
    * max: 633356.0
    * min: 0.0
    * median: 6

In [7]:
def spot_shadow(x):
    if x != None:
        clip_dictionary = x['c']
        if clip_dictionary != None:
            for clip in clip_dictionary:
                if clip!= None and 'ss' in clip.keys():
                    return True
                else:
                    return False
        else:
            return False
    else:
        return False

In [8]:
ss_map = highlight_views.filter(lambda x: spot_shadow(x)).map(lambda x: x['v'])
ss_reduce = ss_map.reduce(lambda a,b: a+b)
ss_views = ss_map.collect()
ss_median = numpy.median(ss_views)
ss_map.stats()

(count: 1668958, mean: 49.2997373211, stdev: 557.941787933, max: 633356.0, min: 0.0)

In [ ]:
nzss_map = ss_map.filter(lambda x: x!=0)
nzss_reduce = nzss_map.reduce(lambda a,b: a+b)
nzss_views = nzss_map.collect()
nzss_median = numpy.median(nzss_views)
nzss_map.stats()

In [ ]:
no_ss = reject_outliers(ss_views)

##No Spot Shadow
* Highlights without spot shadows = 5252045 
* Highlights without spot shadows views = 90433739 
* Highlights without spot shadows information
    * mean: 17.2187669755
    * stdev: 487.332510668
    * max: 1056352.0
    * min: 0.0
    * median: 4.0

In [9]:
nss_map = highlight_views.filter(lambda x: spot_shadow(x) == False).map(lambda x: x['v'])
nss_zero = zero_views.filter(lambda x: spot_shadow(x) == False).count()
nss_reduce = nss_map.reduce(lambda a,b: a+b)
nss_views = nss_map.collect()
nss_median = numpy.median(nss_views)
nss_map.stats()

(count: 5252045, mean: 17.2187669755, stdev: 487.332510668, max: 1056352.0, min: 0.0)

In [41]:
nss_reduce

90433739

##Slow Motion
* Highlights with slow motion = 10
* Highlight with slow motion views = 97
* Highlights with slow motion view information
    * mean: 9.7
    * stdev: 15.9
    * max: 55.0
    * min: 0.0
    * median: 2

In [10]:
def slow_motion(x):
    if x != None:
        clip_dictionary = x['c']
        if clip_dictionary != None:
            for clip in clip_dictionary:
                if clip!= None and 'sm' in clip.keys():
                    return True
                else:
                    return False
        else:
            return False
    else:
        return False

In [11]:
sm_map = highlight_views.filter(lambda x: slow_motion(x)).map(lambda x: x['v'])
sm_reduce = sm_map.reduce(lambda a,b: a+b)
sm_views = sm_map.collect()
sm_median = numpy.median(sm_views)
sm_map.stats()

(count: 10, mean: 9.7, stdev: 15.9, max: 55.0, min: 0.0)

In [12]:
nzsm_map = sm_map.filter(lambda x: x!=0)
nzsm_reduce = nzsm_map.reduce(lambda a,b: a+b)
nzsm_views = nzsm_map.collect()
nzsm_median = numpy.median(nzsm_views)
nzsm_map.stats()

(count: 8, mean: 12.125, stdev: 16.9295414882, max: 55.0, min: 1.0)

In [14]:
no_sm = reject_outliers(sm_views)

##No Slow Motion
* Highlights without slow motion = 6920993 
* Highlight without slow motion views = 172712833 
* Highlights without slow motion view information
    * mean: 24.9549209196 
    * stdev: 505.449863751
    * max: 1056352.0
    * min: 0.0
    * median: 4

In [32]:
nsm_map = highlight_views.filter(lambda x: slow_motion(x) == False).map(lambda x: x['v'])
nsm_zero = zero_views.filter(lambda x: slow_motion(x) == False).count()
nsm_reduce = nsm_map.reduce(lambda a,b: a+b)
nsm_views = nsm_map.collect()
nsm_median = numpy.median(nsm_views)
nsm_map.stats()

(count: 6920993, mean: 24.9549209196, stdev: 505.449863751, max: 1056352.0, min: 0.0)

##Music
* Highlights with music = 500195
* Highlight with music views = 95432835
* Highlights with music view information
    * mean: 190.791261408
    * stdev: 1851.80389818
    * max: 1056352.0
    * min: 0.0
    * median: 77

In [15]:
def highlight_music(x):
        if x!= None and 'hm' in x.keys():
            return True
        else:
            return False

In [16]:
hm_map = highlight_views.filter(lambda x: highlight_music(x)).map(lambda x: x['v'])
hm_reduce = hm_map.reduce(lambda a,b: a+b)
hm_views = hm_map.collect()
hm_median = numpy.median(hm_views)
hm_map.stats()

(count: 500195, mean: 190.791261408, stdev: 1851.80389818, max: 1056352.0, min: 0.0)

In [17]:
nzhm_map = hm_map.filter(lambda x: x!=0)
nzhm_reduce = nzhm_map.reduce(lambda a,b: a+b)
nzhm_views = nzhm_map.collect()
nzhm_median = numpy.median(nzhm_views)
nzhm_map.stats()

(count: 498530, mean: 191.428469701, stdev: 1854.86078467, max: 1056352.0, min: 1.0)

In [18]:
no_hm = reject_outliers(hm_views)

##No Music
* Highlights with music = 6427524 
* Highlight with music views = 95432835
* Highlights with music view information
    * mean: 12.0364564955 
    * stdev: 76.9994446654 
    * max: 71279.0 
    * min: 0.0
    * median: 4.0

In [19]:
nhm_map = highlight_views.filter(lambda x: highlight_music(x) == False).map(lambda x: x['v'])
nhm_zero = zero_views.filter(lambda x: highlight_music(x) == False).count()
nhm_reduce = nhm_map.reduce(lambda a,b: a+b)
nhm_views = nhm_map.collect()
nhm_median = numpy.median(nhm_views)
nhm_map.stats()

(count: 6427524, mean: 12.0364564955, stdev: 76.9994446654, max: 71279.0, min: 0.0)

##Extra Text
* Highlights with extra text = 287203
    * *296086 before removing highlights without views*
    * *3% data lost*
* Highlight with extra text views = 50743514
* Highlights with extra text information
    * mean: 176.681699007
    * stdev: 514.299588065
    * max: 54566.0
    * min: 0.0
    * median: 77

In [20]:
def extra_text(x):
        if x!= None and 'xt' in x.keys():
            return True
        else:
            return False

In [21]:
xt_map = highlight_views.filter(lambda x: extra_text(x)).map(lambda x: x['v'])
xt_reduce = xt_map.reduce(lambda a,b: a+b)
xt_views = xt_map.collect()
xt_median = numpy.median(xt_views)
xt_map.stats()

(count: 287203, mean: 176.681699007, stdev: 514.299588065, max: 54566.0, min: 0.0)

In [22]:
nzxt_map = xt_map.filter(lambda x: x!=0)
nzxt_reduce = nzxt_map.reduce(lambda a,b: a+b)
nzxt_views = nzxt_map.collect()
nzxt_median = numpy.median(nzxt_views)
nzxt_map.stats()

(count: 286925, mean: 176.8528849, stdev: 514.519259163, max: 54566.0, min: 1.0)

In [23]:
no_xt = reject_outliers(xt_views)

#No extra text
* Highlights without extra text = 6640516 
    * *296086 before removing highlights without views*
    * *3% data lost*
* Highlight without extra text views = 122053934 
* Highlights without extra text information
    * mean: 18.3801882263
    * stdev: 503.780672043 
    * max: 1056352.0 
    * min: 0.0
    * median: 4.0

In [24]:
net_map = highlight_views.filter(lambda x: extra_text(x) == False).map(lambda x: x['v'])
net_zero = zero_views.filter(lambda x: extra_text(x) == False).count()
net_reduce = net_map.reduce(lambda a,b: a+b)
net_views = net_map.collect()
net_median = numpy.median(net_views)
net_map.stats()

(count: 6640516, mean: 18.3801882263, stdev: 503.780672043, max: 1056352.0, min: 0.0)

##Top Play
* Highlights with top play = 6429859
    * 6486866 before removing highlights without views
    * .9% data lost
* Highlight with top play views = 164196611
* Highlights with extra text information
    * mean: 25.5365803511
    * stdev: 523.580771891
    * max: 1056352.0
    * min: 0.0
    * median: 4

In [25]:
def top_play(x):
        if x!= None and 'itp' in x.keys():
            return True
        else:
            return False

In [26]:
ytp_map = highlight_views.filter(lambda x: top_play(x)).map(lambda x: x['v'])
ytp_reduce = ytp_map.reduce(lambda a,b: a+b)
ytp_views = ytp_map.collect()
ytp_median = numpy.median(ytp_views)
ytp_map.stats()

(count: 6429859, mean: 25.5365803511, stdev: 523.580771891, max: 1056352.0, min: 0.0)

In [27]:
nzytp_map = ytp_map.filter(lambda x: x!=0)
nzytp_reduce = nzytp_map.reduce(lambda a,b: a+b)
nzytp_views = nzytp_map.collect()
nzytp_median = numpy.median(nzytp_views)
nzytp_map.stats()

(count: 5295463, mean: 31.0070358343, stdev: 576.795363943, max: 1056352.0, min: 1.0)

In [28]:
no_ytp = reject_outliers(ytp_views)

##No Top Play
* Highlights without top play = 497860 
* Highlight with top play views = 8600837 
* Highlights with extra text information
    * mean: 17.2756136263 
    * stdev: 105.124979881 
    * max: 22236.0 
    * min: 0.0
    * median: 3.0

In [29]:
ntp_map = highlight_views.filter(lambda x: top_play(x) == False).map(lambda x: x['v'])
ntp_zero = zero_views.filter(lambda x: top_play(x) == False).count()
ntp_reduce = ntp_map.reduce(lambda a,b: a+b)
ntp_views = ntp_map.collect()
ntp_median = numpy.median(ntp_views)
ntp_map.stats()

(count: 497860, mean: 17.2756136263, stdev: 105.124979881, max: 22236.0, min: 1.0)

##Figures Code using Plotly
###Box Plot

In [ ]:
p_box = Box(
   y=no_yp,
   name='Premium',
)

ss_box = Box(
   y=no_ss,
   name='Spot Shadow',
)

sm_box = Box(
   y=no_sm,
   name='Slow Motion',
)

hm_box = Box(
   y=no_hm,
   name='Highlight Music',
)

et_box = Box(
   y=no_xt,
   name='Extra Text',
)

tp_box = Box(
   y=no_ytp,
   name='Top Play',
)

layout = Layout(
   title='Public Highlight Views for 6 Special Properties',
   showlegend=False,
   yaxis=YAxis(
       title='Public Highlight Views'),
   xaxis=XAxis(
       title='Special Properties'),
   )

traces = Data([p_box, ss_box, sm_box, hm_box, et_box, tp_box])
fig = Figure(data=traces, layout=layout)
py.iplot(fig, filename='basic-box-plot')

##Bar Graph

In [16]:
y_properties_yes = [636552, 1668958, 10, 500195, 287203, 6429859]
y_properties_no = [6291167, 5252045, 6920993, 6427524, 6640516, 497860]
y_views_yes = [114767406,  82279191 , 97, 95432835, 50743514, 164196611]
y_views_no = [58030042, 90433739, 172712833, 77364613, 122053934, 8600837]

x_properties = ['Premium', 'Spot Shadow', 'Slow Motion', 'Music','Extra Text', 'Top Play']
x_views = ['Premium', 'Spot Shadow', 'Slow Motion', 'Music','Extra Text', 'Top Play']

trace1 = Bar(
   x=y_properties_yes,
   y=x_properties,
   orientation='h',
   name = "Public highlights with the special property",
   marker = Marker(color = 'rgba(255,0,0,.7)'),
)

trace2 = Bar(
   x=y_properties_no,
   y=x_properties,
   orientation='h',
   name = "Public highlights without the special property",
   marker = Marker(color = 'rgba(0,0,128,.7)'),
)

trace3 = Bar(x=y_views_yes, 
            y=x_views, 
            orientation='h',
            name = "Public highlight views with the special property",
            marker = Marker(color = 'rgba(0,128,0,.5)')
)

trace4 = Bar(x=y_views_no, 
                y=x_views, 
                orientation='h',
                name = "Public highlight views without the special property",
                marker = Marker(color = 'rgba(128,0,128,.5)')
)

layout = dict(
   title='Public Highlight Analysis',
   yaxis1=YAxis(
       showgrid=False,
       showline=False,
       showticklabels=True,),
   yaxis2=YAxis(
       showgrid=False,
       showline=True,
       showticklabels=False,),
   xaxis1=XAxis(
       zeroline=False,
       showline=False,
       showticklabels=True,
       showgrid=True,
       title = "Number of Highlights with a Property"),
   xaxis2=XAxis(
       zeroline=False,
       showline=False,
       showticklabels=True,
       showgrid=True,
        title = "Number of Total Views with a Property"
   ),
 legend=Legend(
        x=0.001,
        y=1.3,
        font=Font(
            size=10,
        ),
))

#ANNOTATIONS

annotations = []
# Adding labels
for yvy, yvn, ypy, ypn, xp, xv in zip(y_views_yes, y_views_no, y_properties_yes, y_properties_no, x_properties, x_views):
    # labeling the bar graph
    annotations.append(Annotation(xref='x1', yref='y1', y=xp, x=ypy + ypn + 400000,
                                  text=str(int(ypy/float(ypy + ypn) * 100)) + '%',
                                  showarrow=False))
    # labeling the line graph
    annotations.append(Annotation(xref='x2', yref='y2', y=xv, x=yvy + yvn + 10000000,
                                  text=str(int(yvy/float(yvy + yvn) * 100)) + '%',
                                  showarrow=False))

layout['annotations'] = annotations
 
    
fig = tools.make_subplots(rows = 1, cols = 2, horizontal_spacing = .01)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)
fig['layout'].update(layout, barmode='stack')
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



##Questions

1. Premium vs nonpremium? Premium views vs nonpremium views?
2. Spot shadows vs no spot shadows? Spot shadow views vs no spot shadow views?
3. Slow motion vs no slow motion? Slow motion views vs no slow motion views?
4. Music vs no music? Music views vs no music views?
5. Extra text vs no extra text? Extra text views vs no extra text views?
6. Top plays vs non top play? Top play views vs non top play views?
77. 6.Video length vs views? Create line graph. X axis video length and y axis views.

#Answers